<a href="https://colab.research.google.com/github/Ry02024/CustomSearchRankChecker/blob/main/CustomSearchRankChecker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

制作過程：https://chatgpt.com/share/5f9e1948-f7f6-45e3-9fe5-2bb911918d2d

#streamlit

##環境構築

In [ ]:
!pip install streamlit -q
!pip install pyngrok -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.6 MB/s eta 0:00:00


In [ ]:
import streamlit as st
from pyngrok import ngrok

In [ ]:
from google.colab import userdata
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

In [ ]:
from pyngrok import ngrok
import os

# ngrokトークンを設定
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

## streamlit files

In [ ]:
%%writefile fetch.py
import requests

def fetch_search_results(keyword, api_key, cse_id, num_results=10, start=1):
    search_url = "https://www.googleapis.com/customsearch/v1"
    params = {
        'q': keyword,
        'key': api_key,
        'cx': cse_id,
        'num': min(num_results, 10),
        'start': start
    }

    response = requests.get(search_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"検索に失敗しました。ステータスコード: {response.status_code}")
        print(f"エラーメッセージ: {response.text}")
        return None


Overwriting fetch.py


In [ ]:
%%writefile ranking.py
from fetch import fetch_search_results

def check_rankings(keywords, target_url, api_key, cse_id, max_results=30):
    rankings = []
    for keyword in keywords:
        results = []
        for start in range(1, max_results + 1, 10):
            batch_results = fetch_search_results(keyword, api_key, cse_id, num_results=10, start=start)
            if batch_results and 'items' in batch_results:
                results.extend(batch_results['items'])
                if len(batch_results['items']) < 10:
                    break
            else:
                break

        if results:
            found = False
            for index, item in enumerate(results):
                link = item.get('link')
                if target_url in link:
                    rankings.append({
                        "keyword": keyword,
                        "rank": index + 1,
                        "title": item.get('title'),
                        "snippet": item.get('snippet'),
                        "url": link
                    })
                    found = True
                    break
            if not found:
                rankings.append({
                    "keyword": keyword,
                    "rank": None,
                    "title": None,
                    "snippet": None,
                    "url": None
                })
        else:
            rankings.append({
                "keyword": keyword,
                "rank": None,
                "title": None,
                "snippet": None,
                "url": None
            })
    return rankings


Overwriting ranking.py


In [ ]:
%%writefile display.py
import streamlit as st

def display_rankings(rankings):
    for ranking in rankings:
        st.write(f"キーワード: {ranking['keyword']}")
        if ranking['rank'] is not None:
            st.write(f"順位: {ranking['rank']}")
            st.write(f"タイトル: {ranking['title']}")
            st.write(f"スニペット: {ranking['snippet']}")
            st.write(f"リンク: {ranking['url']}")
        else:
            st.write("順位: 圏外")
        st.write("\n")


Overwriting display.py


In [ ]:
%%writefile streamlit_app.py
import streamlit as st
from ranking import check_rankings
from display import display_rankings

st.title("自然検索順位アプリ")

# Step 1: API Key and Custom Search Engine ID
if 'api_key' not in st.session_state:
    st.session_state.api_key = ''
if 'cse_id' not in st.session_state:
    st.session_state.cse_id = ''

if not st.session_state.api_key or not st.session_state.cse_id:
    api_key = st.text_input("Google Search API Key")
    cse_id = st.text_input("Google Engine ID")

    if api_key and cse_id:
        st.session_state.api_key = api_key
        st.session_state.cse_id = cse_id
        st.experimental_rerun()  # 入力後に再読み込みしてAPIキーとIDをセッションに保存
else:
    st.success("APIキーとカスタム検索エンジンIDが設定されました。")

    # Step 2: Target URL and Keywords
    target_url = st.text_input("ターゲットURL", "https://porely.jp/")
    keywords_input = st.text_area("キーワード（カンマ区切り）", "妊娠, 出産, 育児, キャリア, 育休, 医師, 職場 復帰, パートナーシップ")

    if 'max_results' not in st.session_state:
        st.session_state.max_results = 30

    max_results = st.number_input("最大の検索結果の件数", min_value=10, max_value=100, value=st.session_state.max_results, step=10)

    if target_url and keywords_input:
        st.session_state.target_url = target_url
        st.session_state.keywords_input = keywords_input
        st.session_state.max_results = max_results

        if st.button("検索"):
            keywords = [keyword.strip() for keyword in keywords_input.split(",")]
            rankings = check_rankings(keywords, target_url, st.session_state.api_key, st.session_state.cse_id, max_results=max_results)
            display_rankings(rankings)


Overwriting streamlit_app.py


##アプリの起動

In [ ]:
# ngrokを介してStreamlitを公開
public_url = ngrok.connect(addr='8501')
print('Public URL:', public_url)

Public URL: NgrokTunnel: "https://2e50-107-167-188-216.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
# Streamlitアプリケーションの起動
!streamlit run streamlit_app.py >/dev/null

2024-06-02 02:43:04.700 Please replace `st.experimental_rerun` with `st.rerun`.

`st.experimental_rerun` will be removed after 2024-04-01.
